# Laboratorio 2 - Clasificador de rostros

**Autores**

*   [214205] Enrique, Oliva
*   [192680] Martina, Severo
*   [229484] Santiago, Tonarelli

**Formato de entrega**:

* Esta misma notebook en formato .ipynb
* Cambiar el nombre de la notebook por NumEst1_NumEst2_NumEst3_Lab_1.
* Es importante que la notebook pueda ejecutarse sin problemas al seleccionar 'Ejecutar todo'.
* Se considerará que sus datos pueden estar en otra localización.


**Plazo de entrega**: hasta el Domingo 16/06 a las 23:59 horas a través de Aulas.

**Objetivo**: implementar un algoritmo de clasificación que permita predecir si una imagen dada es un rostro o no.

## Librerías

In [ ]:
import os
from tqdm import tqdm
from time import time

import random
import numpy as np
import matplotlib.pyplot as plt

from skimage.exposure import equalize_hist

from skimage.transform import integral_image
from skimage.feature import haar_like_feature, haar_like_feature_coord

from sklearn.feature_selection import SelectPercentile, f_classif

## Funciones auxiliares

In [ ]:
def extract_feature_image(img, feature_type=None, feature_coord=None):
    """Extrae las Haar features de la imagen"""
    ii = integral_image(img)
    return haar_like_feature(ii, 0, 0, ii.shape[0], ii.shape[1],
                             feature_type=feature_type,
                             feature_coord=feature_coord)

## Datos

**CBCL FACE DATABASE #1**:

*   19 x 19 Grayscale PGM format images
*   Training set:  2429 faces, 4548 non-faces
*   Test set: 472 faces, 23573 non-faces



In [ ]:
# !unzip /content/CBCL.zip
!tar -xvzf /content/face.test.tar.gz
!tar -xvzf /content/face.train.tar.gz

Streaming output truncated to the last 5000 lines.
train/face/face01979.pgm
train/face/face01980.pgm
train/face/face01981.pgm
train/face/face01982.pgm
train/face/face01983.pgm
train/face/face01984.pgm
train/face/face01985.pgm
train/face/face01986.pgm
train/face/face01987.pgm
train/face/face01988.pgm
train/face/face01989.pgm
train/face/face01990.pgm
train/face/face01991.pgm
train/face/face01992.pgm
train/face/face01993.pgm
train/face/face01994.pgm
train/face/face01995.pgm
train/face/face01996.pgm
train/face/face01997.pgm
train/face/face01998.pgm
train/face/face01999.pgm
train/face/face02000.pgm
train/face/face02001.pgm
train/face/face02002.pgm
train/face/face02003.pgm
train/face/face02004.pgm
train/face/face02005.pgm
train/face/face02006.pgm
train/face/face02007.pgm
train/face/face02008.pgm
train/face/face02009.pgm
train/face/face02010.pgm
train/face/face02011.pgm
train/face/face02012.pgm
train/face/face02013.pgm
train/face/face02014.pgm
train/face/face02015.pgm
train/face/face02016.pgm

In [ ]:
suffix = '.pgm'

train_faces = os.listdir('/content/train/face')
train_faces = [filename for filename in train_faces if filename.endswith(suffix)]

train_background = os.listdir('/content/train/non-face')
train_background = [filename for filename in train_background if filename.endswith(suffix)]

test_faces = os.listdir('/content/test/face')
test_faces = [filename for filename in test_faces if filename.endswith(suffix)]

test_background = os.listdir('/content/test/non-face')
test_background = [filename for filename in test_background if filename.endswith(suffix)]

In [ ]:
print(f'# Train Faces: {len(train_faces)}')
print(f'# Train Back: {len(train_background)}')
print(f'# Test Faces: {len(test_faces)}')
print(f'# Test Back: {len(test_background)}')

# Train Faces: 2429
# Train Back: 4548
# Test Faces: 472
# Test Back: 23573


In [ ]:
# Tomaremos una fracción de los datos. Puede ajustar estos parámetros a gusto
f = 0.2
n_face = int(f*len(train_faces))
n_back = int(f*len(train_background))

# Para mantener la proporción de background en test calculamos (para mantener una proporción balanceada entre las clases (rostros y no-rostros)):
m = int(np.round(len(test_faces)*len(train_background)/len(train_faces)))

print(f'# Train Faces Sample Size: {n_face}')
print(f'# Train Back Sample Size: {n_back}')
print(f'# m: {m}')

# Train Faces Sample Size: 485
# Train Back Sample Size: 909
# m: 884


In [ ]:
sample_train_faces = random.sample(train_faces,n_face)

Im_train = []
for filename in tqdm(sample_train_faces):
    path = '/content/train/face/' + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_train.append(image)

n_train_faces = len(Im_train)
y_train = [1]*n_train_faces # Cada imagen de rostro se etiqueta con un 1

100%|██████████| 485/485 [00:00<00:00, 5227.27it/s]


In [ ]:
sample_train_background = random.sample(train_background,n_back)

for filename in tqdm(sample_train_background):
    path = "/content/train/non-face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_train.append(image)

n_train_background = len(Im_train)-n_train_faces
y_train = y_train + [0]*n_train_background # Cada imagen de no-rostro se etiqueta con un 0

100%|██████████| 909/909 [00:00<00:00, 7528.07it/s]


In [ ]:
print(f'# Train: {len(Im_train)}, {len(y_train)}')

# Train: 1394, 1394


In [ ]:
Im_test = []
for filename in tqdm(test_faces):
    path = "/content/test/face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_test.append(image)

n_test_faces = len(Im_test)
y_test = [1]*n_test_faces

100%|██████████| 472/472 [00:00<00:00, 5527.64it/s]


In [ ]:
sample_test_background = random.sample(test_background,m)

for filename in tqdm(sample_test_background):
    path = "/content/test/non-face/" + filename
    with open(path, 'rb') as pgmf:
        image = plt.imread(pgmf)
    Im_test.append(image)

n_test_background = len(Im_test)-n_test_faces
y_test = y_test + [0]*n_test_background

100%|██████████| 884/884 [00:00<00:00, 5618.63it/s]


In [ ]:
print(f'# Test: {len(Im_test)}, {len(y_test)}')

# Test: 1356, 1356


## Histogram equalization

In [ ]:
# Normalización de las imágenes de entrenamiento y prueba
Im_train_norm = [equalize_hist(image) for image in Im_train]
Im_test_norm = [equalize_hist(image) for image in Im_test]

## Matriz de features

### Calculamos y seleccionamos las mejores features en entrenamiento

In [ ]:
X_train = [extract_feature_image(img) for img in tqdm(Im_train_norm)]
X_train = np.array(X_train)

100%|██████████| 1394/1394 [04:18<00:00,  5.39it/s]


In [ ]:
# Pueden guardar la matriz si lo desean
np.save('X_train', X_train)

In [ ]:
# Y cargarla posteriormente
X_train = np.load('X_train.npy')

In [ ]:
X_train.shape

(1394, 63960)

In [ ]:
# Selección de características en train
print("Seleccionando las features de mayor dependencia lineal con y")
t_start = time()
# SelectPercentile: selecciona las mejores características basadas en una prueba estadística.
# f_classif: mide la dependencia lineal entre dos conjuntos de datos.
# percentile=1: Selecciona el 1% de las mejores características.
# fit(X_train, y_train): Ajusta el selector de características a los datos de entrenamiento X_train y y_train.
# get_support(indices=True): Obtiene los índices de las características seleccionadas.
f_indices = SelectPercentile(f_classif, percentile=1).fit(X_train, y_train).get_support(indices=True)
t = time() - t_start
X_train = X_train[:,f_indices]
print("Seleccionadas %d features potenciales" % X_train.shape[1])
print(f'Tiempo: {t} segundos')

Seleccionando las features de mayor dependencia lineal con y
Seleccionadas 640 features potenciales
Tiempo: 1.2167468070983887 segundos


### Calculamos dichas features para test

In [ ]:
# haar_like_feature_coord(): genera coordenadas y tipos de características Haar para una ventana de búsqueda especificada. En este caso 19x19 píxeles.
feature_coord, feature_type = haar_like_feature_coord(width=19,
                                                      height=19,
                                                      )

In [ ]:
t_start = time()
X_test = [extract_feature_image(img,
                                feature_type=feature_type[f_indices],
                                feature_coord=feature_coord[f_indices]) for img in tqdm(Im_test_norm)]
t = time() - t_start
X_test = np.array(X_test)

100%|██████████| 1356/1356 [00:01<00:00, 861.07it/s]


In [ ]:
print(f'Tiempo: {t} segundos')
print(f'Shape X_test: {X_test.shape}')

Tiempo: 1.5834944248199463 segundos
Shape X_test: (1356, 640)


In [36]:
# Entrenamiento del Modelo Random Forest
from sklearn.ensemble import RandomForestClassifier

# Crear el clasificador Random Forest
rf_classifier = RandomForestClassifier(n_estimators=600, random_state=42)

# Entrenar el clasificador con el conjunto de entrenamiento y las características extraídas
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(n_estimators=600, random_state=42)

In [37]:
# Evaluación del Modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predecir las etiquetas de las imágenes de prueba
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))


Precisión del modelo: 0.7588495575221239

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.74      0.98      0.84       884
           1       0.88      0.35      0.51       472

    accuracy                           0.76      1356
   macro avg       0.81      0.66      0.67      1356
weighted avg       0.79      0.76      0.72      1356


Matriz de confusión:
[[862  22]
 [305 167]]
